In [ ]:
import urllib.request
filename = 'ratebeer.json.gz'
urllib.request.urlretrieve('https://datarepo.eng.ucsd.edu/mcauley_group/data/beer/ratebeer.json.gz', filename)

In [ ]:
from dask.distributed import Client, LocalCluster, default_client

try:
    if default_client() is not None:
        default_client().close()
except ValueError:
    pass

cluster = LocalCluster(n_workers=8)
client = Client(cluster)
dashboard_link = client.dashboard_link
print ("Dask Dashboard link: ", dashboard_link)

In [ ]:
import gzip
import shutil

def unzip_gzip(input_file, output_file):
    with gzip.open(input_file, 'rb') as f_in:
        with open(output_file, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

unzip_gzip('ratebeer.json.gz', 'data.json')

In [ ]:
import json

def json_to_jsonlines(input_file):
   jsonHolder =  []
   with open(input_file, 'r') as input_file:
     for obj in input_file:
       try:
         data_dict = json.loads(obj.replace("'", "\""))
         jsonHolder.append({"review/profileName": data_dict["review/profileName"], "rating": data_dict['review/overall'], "beer/beerId": data_dict["beer/beerId"]})
       except:
         pass
   return jsonHolder

input_file = 'data.json'
jsonHolder = json_to_jsonlines(input_file)

In [ ]:
import json

def get_beer_with_names(input_file):
   jsonHolder =  {}
   with open(input_file, 'r') as input_file:
     for obj in input_file:
       try:
         data_dict = json.loads(obj.replace("'", "\""))
         jsonHolder[data_dict["beer/beerId"]] = data_dict["beer/name"]
       except:
         pass
   return jsonHolder

input_file = 'data.json'
beer_with_name = get_beer_with_names(input_file)

In [ ]:
def convert_to_jsonl():
    with open('data.jsonl', 'w') as f:
      for entry in jsonHolder:
        json.dump(entry, f)
        f.write('\n')

convert_to_jsonl()

In [1]:
from dask.distributed import Client, LocalCluster, default_client

try:
    if default_client() is not None:
        default_client().close()
except ValueError:
    pass

cluster = LocalCluster(n_workers=8)
client = Client(cluster)
dashboard_link = client.dashboard_link
print ("Dask Dashboard link: ", dashboard_link)

/csse/users/jbr257/.local/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33679 instead
  warnings.warn(


Dask Dashboard link:  http://127.0.0.1:33679/status


In [3]:
import dask
from dask import bag as db
import dask.dataframe as dd
import pandas as pd
import json
import numpy as np
data_bag = db.read_text('data.jsonl', blocksize="10MB")
data_bag = data_bag.map(json.loads)

In [4]:
unique_profile_names = data_bag.pluck('review/profileName').distinct().compute()
unique_beer_ids = data_bag.pluck("beer/beerId").distinct().compute()

num_profiles = len(unique_profile_names)
num_beers = len(unique_beer_ids)


In [5]:
user_index_map = {user: idx for idx, user in enumerate(unique_profile_names)}
beer_index_map = {beer: idx for idx, beer in enumerate(unique_beer_ids)}


In [6]:
import dask.array as da


utility_matrix = da.zeros((num_profiles, num_beers))

print(utility_matrix.shape)

(28437, 109279)


In [10]:
def update_utility(row):
    print(row)
    profile_id = user_index_map[row['review/profileName']]
    beer_id = beer_index_map[row['beer/beerId']]
    rating = int(row['rating'].split('/')[0])
    # utility_matrix[profile_id, beer_id] = rating
    # rating = float(row['rating'].split('/')[0])  # Assuming rating is a string like '13/20', taking the numerator
    # utility_matrix[profile_idx, beer_idx] = rating

# utility_matrix = utility_matrix.compute()
data_bag.map_partitions(update_utility).compute()

2024-05-31 01:06:30,522 - distributed.worker - WARNING - Compute Failed
Key:       ('bag-from-delayed-decode-loads-read-block-update_utility-a38a2e5d53266acfd4b7eea5c7661367', 7)
Function:  execute_task
args:      ((subgraph_callable-64f279b5e56d2157ac2d97c96ae6f4ad, (<function map_chunk at 0x7fb44b715b40>, <function loads at 0x7fb46910dfc0>, [(<function decode at 0x7fb44b716d40>, (<function read_block_from_file at 0x7fb44b716c20>, <OpenFile '/csse/users/jbr257/Desktop/BeerRecomender/data.jsonl'>, 72164853, 10309265, b'\n'), 'utf-8', 'strict', None)], None, {})))
kwargs:    {}
Exception: 'TypeError("\'_MapChunk\' object is not subscriptable")'



TypeError: '_MapChunk' object is not subscriptable

2024-05-31 01:06:30,756 - distributed.worker - WARNING - Compute Failed
Key:       ('bag-from-delayed-decode-loads-read-block-update_utility-a38a2e5d53266acfd4b7eea5c7661367', 5)
Function:  execute_task
args:      ((subgraph_callable-64f279b5e56d2157ac2d97c96ae6f4ad, (<function map_chunk at 0x7f720acecf70>, <function loads at 0x7f7228935fc0>, [(<function decode at 0x7f720acee170>, (<function read_block_from_file at 0x7f720acee050>, <OpenFile '/csse/users/jbr257/Desktop/BeerRecomender/data.jsonl'>, 51546324, 10309264, b'\n'), 'utf-8', 'strict', None)], None, {})))
kwargs:    {}
Exception: 'TypeError("\'_MapChunk\' object is not subscriptable")'

2024-05-31 01:06:30,905 - distributed.worker - WARNING - Compute Failed
Key:       ('bag-from-delayed-decode-loads-read-block-update_utility-a38a2e5d53266acfd4b7eea5c7661367', 6)
Function:  execute_task
args:      ((subgraph_callable-64f279b5e56d2157ac2d97c96ae6f4ad, (<function map_chunk at 0x7fcbc8bd4160>, <function loads at 0x7fcbe2fd5fc0>, [(

In [ ]:
print(utility_matrix[0])

dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>
dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>
dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>
dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>
dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>
dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>
dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>


dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.n

dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>
dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>
dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>
dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>
dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>
dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>
dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>
dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>
dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.nd